# Multi-Object Optimization

This notebook allows the identification of paths that balance multiple objectives, e.g., distance, comfort, safety, for pedestrians and cyclists.

The graphs used are created using Neo4j through the code in the GitHub repository: [https://github.com/federicarollo/GRAFMOVE](https://github.com/federicarollo/GRAFMOVE).

Then, each graph was extracted as a collection of two CSV files, available in the **[data folder](https://github.com/federicarollo/ITADATA25/data)**, using the following Cypher queries:

- `MATCH (n) RETURN id(n) AS id, labels(n) AS labels, apoc.convert.toJson(properties(n)) AS properties` for nodes
- `MATCH (n)-[r]->(m) RETURN id(n) as source, id(m) as target, type(r) as type, apoc.convert.toJson(properties(r)) as properties` for edges

The graph structure follows the OpenStreetMap (OSM) structure:
- **Nodes** represent OSM-defined junctions or road shape points, mantaining the OSM identifier and the GPS coordinates as properties,
- **Edges** correspond to roads, with properties that describe their shape, length, type. The direction of the edge indicates the permitted travel direction.

<img src="https://raw.githubusercontent.com/federicarollo/ITADATA25/main/data/images/osm_structure_and_green.png" style="display: block; margin-left: auto; margin-right: auto;" alt="OSM structure" width="700"/>


<div>
  <br><br>    
  <p style="float: left; width: 50%;">
    <b>FootNodes</b> represent nodes where pedestrian access is allowed, whereas <b>BikeNodes</b> correspond to nodes for cyclists.<br>ROUTE edges represent the roads.<br><br>The graph is enriched by the localization of the Point Of Interests (POIs) represented by the <b>POI</b> nodes, and linked to the corresponding elements in OSM (<b>OSMNode</b> or <b>OSMWay</b> if they are stored in OSM as node or way, respectively). The <b>Tag</b> nodes store the characteristics of the POIs (e.g., name, type such touristic attraction or parking).
  </p>
  <img src="https://raw.githubusercontent.com/federicarollo/ITADATA25/main/data/images/graph structure.png" style="display: block; margin-left: auto; margin-right: auto;" alt="Graph structure" style="float: right; width: 40%;">
</div>
<div style="clear: both;"></div>

## Configuration parameters

In [1]:
# nodes_filename = "../data/graphs/Modena/modena_nodes.csv"
# edges_filename = "../data/graphs/Modena/modena_edges.csv"

nodes_filename = "https://raw.githubusercontent.com/federicarollo/ITADATA25/main/data/graphs/Modena/modena_nodes.csv"
edges_filename = "https://raw.githubusercontent.com/federicarollo/ITADATA25/main/data/graphs/Modena/modena_edges.csv"

## Import libraries

Uncomment the following two cells if you need to install libraries

In [5]:
# !pip install torch_geometric

In [6]:
# !pip install pymoo

In [3]:
import networkx as nx
import numpy as np
import pandas as pd
import json

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.indicators.hv import HV
from pymoo.visualization.scatter import Scatter
from pymoo.termination.default import DefaultMultiObjectiveTermination

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import folium as fo

In [4]:
import ast

def double_unescape_json(s):
    s = s.replace("\\n", "")
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        try:
            unescaped = s.encode('utf-8').decode('unicode_escape')
            return json.loads(unescaped)
        except json.JSONDecodeError:
            print(f"Warning: Could not parse properties string: {s}")
            return {}

## Step 1 – Graph Import

In [8]:
nodes_df = pd.read_csv(nodes_filename)
edges_df = pd.read_csv(edges_filename)

In [9]:
nodes_df.shape

(60949, 3)

### Time-dependent properties vs time invariant properties

In [11]:
edges_df['properties'][233]

'{\\"crash_risk\\":[0.2,1.1317460317460317,0.0,0.3713675213675213],\\"distance\\":139.06596543109077,\\"green_area\\":0,\\"green_area_weight\\":139.06596543109077,\\"length\\":\\"138.9104844602295\\",\\"maxspeed\\":\\"70\\",\\"crash_risk_density\\":[8.373572785294012E-6,4.7383788856465315E-5,0.0,1.5548364851325846E-5],\\"oneway\\":\\"True\\",\\"ref\\":\\"SS724\\",\\"crash_risk_per_meter\\":[0.0011644789834944196,0.00658947234310731,0.0,0.002162248368924467],\\"crash_risk_density_norm\\":[0.060809186016005864,0.1828168482443171,0.0,0.04031274398437343],\\"name\\":\\"Tangenziale Sud Gabriela Mistral\\",\\"lanes\\":\\"2\\",\\"geometry\\":\\"LINESTRING(10.8764911 44.6444336, 10.8759706 44.6456267)\\",\\"highway\\":\\"trunk\\",\\"reversed\\":\\"False\\"}'

In [12]:
double_unescape_json(edges_df['properties'][233])

{'crash_risk': [0.2, 1.1317460317460317, 0.0, 0.3713675213675213],
 'distance': 139.06596543109077,
 'green_area': 0,
 'green_area_weight': 139.06596543109077,
 'length': '138.9104844602295',
 'maxspeed': '70',
 'crash_risk_density': [8.373572785294012e-06,
  4.7383788856465315e-05,
  0.0,
  1.5548364851325846e-05],
 'oneway': 'True',
 'ref': 'SS724',
 'crash_risk_per_meter': [0.0011644789834944196,
  0.00658947234310731,
  0.0,
  0.002162248368924467],
 'crash_risk_density_norm': [0.060809186016005864,
  0.1828168482443171,
  0.0,
  0.04031274398437343],
 'name': 'Tangenziale Sud Gabriela Mistral',
 'lanes': '2',
 'geometry': 'LINESTRING(10.8764911 44.6444336, 10.8759706 44.6456267)',
 'highway': 'trunk',
 'reversed': 'False'}

In [13]:
double_unescape_json(edges_df['properties'][233])['distance']

139.06596543109077

In [14]:
double_unescape_json(edges_df['properties'][233])['crash_risk_density_norm']

[0.060809186016005864, 0.1828168482443171, 0.0, 0.04031274398437343]

### Load the graph

In [16]:
%%time

G = nx.DiGraph()

for _, row in nodes_df.iterrows():
    properties = double_unescape_json(row['properties'])
    G.add_node(row['id'], labels=row['labels'], **properties)

for _, row in edges_df.iterrows():
    properties = double_unescape_json(row['properties'])
    G.add_edge(row['source'], row['target'], label=row['type'], **properties)

CPU times: total: 30 s
Wall time: 30.7 s


In [17]:
print(f"Nodes: {len(G.nodes())}, Edges: {len(G.edges())}")

Nodes: 60949, Edges: 166378


### What are the properties of the FootNode instances?

In [19]:
node_properties = set()

for _, data in G.nodes(data=True):
    if 'FootNode' in data.get('labels', []):
        # Aggiungi tutte le chiavi tranne 'labels'
        node_properties.update(k for k in data.keys() if k != 'labels')

print(f"Properties of FootNode instances: {node_properties}")

Properties of FootNode instances: {'longitude', 'location', 'geometry', 'ref', 'x', 'lat', 'componentId', 'pedestrian_allowed_grafmove', 'id', 'railway', 'cyclist_allowed_grafmove', 'highway', 'lon', 'bbox', 'street_count', 'green_area', 'gtype', 'y', 'latitude'}


### What are the properties of the ROUTE edge instances?

In [21]:
edge_property = set()

for u, v, data in G.edges(data=True):
    if data.get('label') == 'ROUTE':
        edge_property.update(k for k in data.keys() if k != 'labels')

print(f"Properties of ROUTE instances: {edge_property}")

Properties of ROUTE instances: {'crash_risk', 'geometry', 'ref', 'foot_class', 'tunnel', 'crash_risk_per_meter', 'label', 'reversed', 'crs', 'distance', 'access', 'maxspeed', 'created_date', 'created_with', 'service', 'crash_risk_density_norm', 'bridge', 'width', 'length', 'highway', 'green_area', 'name', 'green_area_weight', 'oneway', 'bike_class', 'junction', 'lanes', 'crash_risk_density'}


### What are all the possible values of the *highway* property of ROUTE?

In [23]:
highway_values = set()

for _, _, data in G.edges(data=True):
    if data.get('label') == 'ROUTE' and 'highway' in data:
        highway_values.add(data['highway'])

print(highway_values)

{'trunk', 'path', 'secondary', 'busway', 'trunk_link', 'primary', 'cycleway', 'elevator', 'living_street', 'pedestrian', 'service', 'steps', 'motorway', 'motorway_link', 'residential', 'tertiary', 'footway', 'unclassified', 'track', 'tertiary_link', 'secondary_link', 'primary_link'}


### Road length values

In [25]:
route_distances = [
    data['distance']
    for u, v, data in G.edges(data=True)
    if data.get('label') == 'ROUTE' and 'distance' in data
]

In [ ]:
plt.boxplot(route_distances)
plt.title("Boxplot of Route Distances")
plt.ylabel("Distance")
plt.grid(True)
plt.show()

## Step 2 - Path generation

### Configuration parameters

In [26]:
OBJS_NOTIME = ["distance", "green_area_weight"]
OBJS_TIME = ["crash_risk_density_norm"] # or pm25_per_meter
TIME_INTERVAL = 0 # one of the following values: {NIGHT: 0, MORNING: 1, AFTERNOON: 2, EVENING: 3}

N_OBJS = len(OBJS_NOTIME+OBJS_TIME)
OBJS = OBJS_NOTIME + OBJS_TIME

N_CANDIDATE_PATHS = 1000

### Subgraph definition: pedestrians or cyclists?

In [28]:
%%time

foot_nodes = [
    n for n, data in G.nodes(data=True)
    if "labels" in data and "FootNode" in data["labels"]
]

route_edges = [
    (u, v) for u, v, data in G.edges(data=True)
    if "label" in data and data["label"] == "ROUTE"
    and u in foot_nodes and v in foot_nodes
]

H = G.edge_subgraph(route_edges).copy()

CPU times: total: 1min 8s
Wall time: 1min 10s


In [29]:
print(f"Nodes: {len(H.nodes())}, Edges: {len(H.edges())}")

Nodes: 40523, Edges: 86039


### Generate candidate paths

The *get_candidate_paths* function takes as input:
- the graph H
- the OpenstreetMap identifiers (as string) of the source and target nodes (these nodes can be visualized in OSM by replacing the identifier in a link like https://www.openstreetmap.org/node/2093992765)
- the number of paths to generate (optional)

and provides as output a set of *num_paths* paths between source and target

In [32]:
def get_candidate_paths(G, source, target, num_paths=10):
    paths = []
    
    for node, data in G.nodes(data=True):
        if 'FootNode' in data.get('labels', []) and str(data.get("id")) == source:
            source_node = node
            print("Source node found:", node)
            break
    else:
        print("Source node not found.")

    for node, data in G.nodes(data=True):
        if 'FootNode' in data.get('labels', []) and str(data.get("id")) == target:
            target_node = node
            print("Target node found:", node)
            break
    else:
        print("Target node not found.")
    
    for obj in OBJS_NOTIME:
        paths.append(nx.shortest_path(G, source_node, target_node, weight=obj))
    for obj in OBJS_TIME:
        paths.append(nx.shortest_path(G, source_node, target_node, weight=obj[TIME_INTERVAL]))
    for _ in range(num_paths - N_OBJS):
        random_path = nx.shortest_path(G, source_node, target_node, weight=lambda u, v, d: torch.rand(1).item())
        paths.append(random_path)
        
    return paths

Choose one of these exemplar origin-destination points:

In [34]:
%%time


# ---------- FERRARA ---------- 
# s, t = "1150817556", "2093992765"
# s, t ="2711436174", "2093992765"
# s, t ="958004696", "259040297"
# s, t ="2211349960", "1836899403"


# ---------- MODENA ---------- 
s, t ="10053840073", "2041913868"
# s, t ="250846426", "256411970"
# s, t ="250850846", "2021402066"


paths = get_candidate_paths(H, source=s, target=t, num_paths=N_CANDIDATE_PATHS)

Source node found: 39769
Target node found: 17788
CPU times: total: 2min 55s
Wall time: 3min


In [35]:
print(f"Number of paths: {len(paths)}")

Number of paths: 1000


In [36]:
no_duplicates = []

for element in paths:
    if(element not in no_duplicates):
        no_duplicates.append(element)

In [37]:
print(f"Number of non duplicated paths: {len(no_duplicates)}")

Number of non duplicated paths: 944


In [38]:
def evaluate_path(G, path):
    eval_objs = {}

    for obj in OBJS_NOTIME:
        eval_objs[obj] = 0
    for obj in OBJS_TIME:
        eval_objs[obj] = 0

    for i in range(len(path)-1):
        u, v = path[i], path[i+1]
        
        for obj in OBJS_NOTIME:
            eval_objs[obj] += G.edges[u, v][obj]
        for obj in OBJS_TIME:
            value = float(G.edges[u, v][obj][0])
            eval_objs[obj] += value
        
    return eval_objs

path_data = []
for path in no_duplicates:
    eval_objs = evaluate_path(H, path)
    eval_objs["path"] = path
    path_data.append(eval_objs)

In [39]:
print(f"Number of evaluated paths: {len(path_data)}")

Number of evaluated paths: 944


## Step 3 - Pareto Front identification

In [41]:
class RoutingProblem(Problem):
    def __init__(self, path_data):
        self.path_data = path_data
        super().__init__(n_var=1, n_obj=N_OBJS, n_constr=0, xl=0, xu=len(path_data) - 1)

    def _evaluate(self, X, out, *args, **kwargs):
        objs = []
        for i in X:
            idx = int(i[0])
            objs.append([self.path_data[idx][obj] for obj in OBJS_NOTIME+OBJS_TIME])
        out["F"] = np.array(objs)

We use the [**NSGA-II: Non-dominated Sorting Genetic Algorithm**](https://pymoo.org/algorithms/moo/nsga2.html) to identify the Pareto front.

In [43]:
%%time

termination = DefaultMultiObjectiveTermination(
    xtol=1e-8,
    cvtol=1e-6,
    ftol=0.0025,
    period=30,
    n_max_gen=1000,
    n_max_evals=100000
)

problem = RoutingProblem(path_data)
algorithm = NSGA2(pop_size=10000)
res = minimize(problem, algorithm, verbose=True, termination=termination)

pareto_front = res.F
pareto_solutions = [path_data[int(idx)] for idx in res.X]

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |    10000 |    219 |             - |             -
     2 |    20000 |    436 |  0.000000E+00 |             f
     3 |    30000 |    717 |  0.000000E+00 |             f
     4 |    40000 |    973 |  0.000000E+00 |             f
     5 |    50000 |   1248 |  0.000000E+00 |             f
     6 |    60000 |   1567 |  0.000000E+00 |             f
     7 |    70000 |   1887 |  0.000000E+00 |             f
     8 |    80000 |   2281 |  0.000000E+00 |             f
     9 |    90000 |   2703 |  0.000000E+00 |             f
    10 |   100000 |   3167 |  0.000000E+00 |             f
CPU times: total: 1min 47s
Wall time: 1min 49s


<timed exec>:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)


In [44]:
print(f"Number of solutions in Pareto front: {len(pareto_solutions)}")

Number of solutions in Pareto front: 3167


### Visualize the first path in the list on a map

In [46]:
def get_coordinates(path):
    coordinates = []
    for node in path:
        n = G.nodes[node]
        coordinates.append([n['lat'], n['lon']])
    return coordinates

In [47]:
for p_sol in pareto_solutions:
    
    coordinates = get_coordinates(p_sol['path'])
    
    m = fo.Map(location=[coordinates[0][0], coordinates[0][1]], zoom_start=13)
    fo.PolyLine(coordinates, color="green", weight=3).add_to(m)
    m.save("map.html")
    break

In [ ]:
%matplotlib inline

x = pareto_front[:, 0]  # first objective
y = pareto_front[:, 1]  # second objective
z = pareto_front[:, 2]  # third objective

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(x, y, z, cmap='viridis', marker='o')

ax.set_xlabel('First objective')
ax.set_ylabel('Second objective')
ax.set_zlabel('Third objective')

plt.title('Plot 3D dei dati')
plt.show()

In [48]:
def normalize_objectives(F):
    F_min = F.min(axis=0)
    F_max = F.max(axis=0)
    return (F - F_min) / (F_max - F_min + 1e-9), F_min, F_max


def calculate_normalized_hypervolume(F, ref_point=None, verbose=True):
    F_norm, F_min, F_max = normalize_objectives(F)

    if ref_point is None:
        ref_point = np.ones(F.shape[1]) * 1.1

    hv = HV(ref_point=ref_point)
    hv_value = hv.do(F_norm)

    print(f"Normalized hypervolume: {hv_value:.6f}")
    print(f"Ref point: {ref_point}")

    return hv_value, F_norm


def plot_normalized_pareto(F_norm):

    if F_norm.shape[1] == 2:
        plt.scatter(F_norm[:, 0], F_norm[:, 1])
        plt.xlabel("Objective 1 (normalized)")
        plt.ylabel("Objective 2 (normalized)")
    elif F_norm.shape[1] == 3:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(F_norm[:, 0], F_norm[:, 1], F_norm[:, 2])
        ax.set_xlabel("Objective 1 (normalized)")
        ax.set_ylabel("Objective 2 (normalized)")
        ax.set_zlabel("Objective 3 (normalized)")
    else:
        print("Plot only for 2 or 3 objectives.")
        return
    plt.title("Normalized Pareto Front")
    plt.grid(True)
    plt.show()

In [55]:
hv_value, F_norm = calculate_normalized_hypervolume(res.F)
plot_normalized_pareto(F_norm)

Normalized hypervolume: 1.014170
Ref point: [1.1 1.1 1.1]


In [56]:
F_norm

array([[0.01911269, 0.06335048, 0.80237673],
       [0.35861156, 0.38753803, 0.14197273],
       [0.5420919 , 0.56274345, 0.04503945],
       ...,
       [1.        , 1.        , 0.        ],
       [0.        , 0.04509977, 1.        ],
       [0.47401952, 0.49774112, 0.05492109]])

## Step 4 - Optimal path selection and interactive exploration

### 4.1 Find the best solution based on user preferences

In [81]:
OBJS_NOTIME

['distance', 'green_area_weight']

In [83]:
OBJS_TIME

['crash_risk_density_norm']

In [112]:
USER_PREF = [0.1, 0.3, 0.6] # sum must be 1

In [114]:
weights = np.array(USER_PREF)
score = (F_norm * weights).sum(axis=1)
best_idx = score.argmin()

In [116]:
best_idx

8

In [118]:
pareto_solutions[best_idx]

{'distance': 3390.145536455731,
 'green_area_weight': 3390.145536455731,
 'crash_risk_density_norm': 14.485181271556108,
 'path': [39769,
  39768,
  15261,
  15260,
  15259,
  15258,
  15257,
  31128,
  36172,
  15256,
  15255,
  15254,
  15253,
  15252,
  15251,
  11554,
  11586,
  11563,
  11572,
  11555,
  16318,
  11541,
  11534,
  16642,
  12310,
  44579,
  44580,
  12306,
  11556,
  12307,
  12308,
  12311,
  16589,
  16590,
  16591,
  44166,
  16592,
  16593,
  25142,
  16598,
  16599,
  44138,
  16600,
  16603,
  572,
  583,
  575,
  573,
  12341,
  24621,
  12340,
  12363,
  31838,
  12362,
  12368,
  46341,
  46342,
  46343,
  46344,
  46346,
  46348,
  46347,
  8411,
  8410,
  34260,
  34258,
  11432,
  8409,
  7083,
  43833,
  46123,
  25192,
  25193,
  46111,
  17298,
  35862,
  17552,
  17297,
  620,
  619,
  34346,
  16532,
  618,
  46112,
  617,
  26526,
  16522,
  46113,
  6920,
  16524,
  45326,
  45325,
  45324,
  45174,
  45175,
  45176,
  45173,
  34681,
  26519,
 

In [120]:
coordinates = get_coordinates(pareto_solutions[best_idx]['path'])

m = fo.Map(location=[coordinates[0][0], coordinates[0][1]], zoom_start=13)
fo.PolyLine(coordinates, color="green", weight=3).add_to(m)
m.save("best_path_user_preferences.html")

### 4.2 Find the solution with the optimal trade-off (points closest to the center of pareto front)

In [96]:
from numpy.linalg import norm

distances = norm(F_norm, axis=1)
best_idx = distances.argmin()

In [98]:
pareto_solutions[best_idx]

{'distance': 3005.7895052060085,
 'green_area_weight': 3005.7895052060085,
 'crash_risk_density_norm': 17.804426013031385,
 'path': [39769,
  39768,
  15261,
  15260,
  15259,
  15258,
  33149,
  4471,
  31125,
  34609,
  31124,
  31130,
  12388,
  12389,
  4482,
  16261,
  28349,
  4479,
  12326,
  4480,
  12329,
  12330,
  42371,
  26126,
  11029,
  11030,
  16341,
  12352,
  12353,
  34370,
  25154,
  13855,
  31790,
  37012,
  16615,
  12356,
  12224,
  10954,
  5582,
  10955,
  10956,
  16356,
  571,
  34475,
  574,
  26136,
  12364,
  12363,
  31838,
  12362,
  12368,
  46341,
  46342,
  46343,
  46344,
  46346,
  46348,
  46347,
  8411,
  8410,
  34260,
  34258,
  11432,
  8409,
  7083,
  43833,
  46123,
  25192,
  25193,
  8333,
  8332,
  621,
  35858,
  17551,
  16550,
  620,
  619,
  34346,
  16532,
  618,
  46112,
  617,
  26526,
  16522,
  46113,
  6920,
  26572,
  13890,
  616,
  46114,
  615,
  45174,
  45175,
  36359,
  36360,
  26518,
  46116,
  34681,
  26519,
  24170,

In [100]:
coordinates = get_coordinates(pareto_solutions[best_idx]['path'])

m = fo.Map(location=[coordinates[0][0], coordinates[0][1]], zoom_start=13)
fo.PolyLine(coordinates, color="green", weight=3).add_to(m)
m.save("optimal_trade_off.html")